In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch

from celluloid import Camera
from IPython.display import HTML
from base64 import b64encode

In [2]:
def gaussian(x, mu, sig):
    norm = math.sqrt(2 * math.pi) * sig
    return np.exp(-((x - mu) * (x - mu)) / (2 * sig * sig)) / norm

def construct_operators(N, alpha, beta):
    ST, SV = np.zeros((2, N, N)) # both are [N x N] matrices
    temp = np.arange(N)
    cutoff = N//1
    wall = int(N//1)
    for l in range(N):
        ST[:, l] += temp
        ST[l, :] -= np.transpose(temp)
        SV[:, l] += V(temp, N)
        SV[l, :] -= V(np.transpose(temp), N)
    
    ST *= alpha * ST
    SV *= beta

    ST = torch.tensor(ST) * 1
    SV = torch.tensor(SV) * 1

    K = (torch.complex(0*ST, ST) - torch.complex(0*SV, SV)).exp()

    return ST, SV, K

def normalize(psi):
    return psi / psi.norm()

class ObjectView(object):
    def __init__(self, d): self.__dict__ = d

### I have changed dx and dt to atomic phys natural units
### The numbers are now prop to hbar/(m_e c) and hbar/(m_e c^2), respectively
def init(hprms):
    constants = ObjectView(dict(
                             hbar=1/hprms.scale, m=hprms.m, 
                             numdx=int(hprms.spatial_density*hprms.scale**1.5),
                             numdt=int(hprms.time_density*hprms.scale**1.5), xrange=int(hprms.xrange), 
                             trange=int(hprms.trange), mom=hprms.momentum
                            ))
    
    dx = constants.xrange / constants.numdx  # scalar
    dt = constants.trange / constants.numdt  # scalar
    alpha = constants.m * (dx ** 2 / (2 * constants.hbar * dt))  # scalar
    beta = dt / constants.hbar  # scalar
    steps = constants.numdt // constants.trange
    interval = 2000 * hprms.vid_length / constants.numdt
    
    x = np.linspace(0, constants.xrange, constants.numdx)
    t = np.linspace(0, constants.trange, constants.numdt)

    ST, SV, K = construct_operators(constants.numdx, alpha, beta) # all three are [numdx x numdx]

    r0 = gaussian(x, constants.xrange/2, .025*constants.xrange)[:,None] # r0: [N x 1]
    r0[0, 0] = r0[-1, 0] = 0
    q0 = np.concatenate((np.cos(constants.mom * x)[:,None] * r0,
                         np.sin(constants.mom * x)[:,None] * r0), axis=1)
    q = torch.view_as_complex(torch.tensor(q0))
    
    
    return ObjectView(dict(x=x, t=t, ST=ST, SV=SV, K=K, q=q, interval=interval, steps=steps))

def simulate(q, K, steps=500):
    q = normalize(q)
    qs = [q.clone()]
    for i in range(steps):
        q = normalize(q @ K)
        qs.append(q.clone())
    return torch.stack(qs)

def make_video(x, qs, title, path, interval=60, **kwargs):
    plt.title(title, fontweight="bold")
    plt.ylabel('Sqrt of Probability Density')
    fig = plt.gcf() ; fig.set_dpi(100) ; fig.set_size_inches(5, 3)
    camera = Camera(fig)
    
    for i, q in enumerate(qs):
        plt.plot(x, q.abs().detach(), 'b', label='Spatial probability')
        camera.snap()
    anim = camera.animate(blit=True, interval=interval, **kwargs)
    anim.save(path)
    plt.close()
    
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    HTML('<video width=400 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

def make_static(lft, lftnm, rgt, rgtnm, path):
    fig = plt.figure(figsize=(6, 3), dpi=300)
    ax1 = fig.add_subplot(121)
    ax1.imshow(lft) ; ax1.title.set_text(lftnm)
    ax2 = fig.add_subplot(122)
    ax2.imshow(rgt) ; ax2.title.set_text(rgtnm)
    plt.tight_layout()
    plt.show()
    if path != 'show':
        fig.savefig(path)
    plt.close()

# Pass a valid path if you want to save plots, or False to skip relevant computes
def produce_graphs(hprms, video=False, static_K='show', static_S='show'):
    trial_vars = init(hprms)
    if video:
        title = f'1D Gaussian wave packet, scale = {hprms.scale}'
        qs = simulate(trial_vars.q, trial_vars.K, steps=trial_vars.steps)
        make_video(trial_vars.x, qs, title, path=video, interval=trial_vars.interval, )        
    if static_K:
        make_static(trial_vars.K.real, 'K (real)', trial_vars.K.imag, 'K (imag)', path=static_K)
    if static_S:
        make_static(trial_vars.ST, 'ST', trial_vars.SV, 'SV', path=static_S)

In [ ]:
def V(x, N):
    div = 2; plat=int(N/2 - N/5)
    pot = np.linspace(-N/div, N/div, N) ** 2
    #pot[plat:int(N/2)] = pot[plat:int(N/2)] * 0
    #pot[int(N/2):-plat] = pot[int(N/2):-plat] * 0
    return pot*0

scales = [2, 2.5, 3]
for item in scales:
    hprms=ObjectView(dict(scale=item, m=1, spatial_density=10000,
                          time_density=500, xrange=8, trange=2,
                          momentum=15, vid_length=15))

    produce_graphs(hprms, video=f'quantum_plots/0V/{hprms.scale}scale.mp4',
                   static_S=f'quantum_plots/0V/S_{hprms.scale}scale.png', 
                   static_K=f'quantum_plots/0V/K_{hprms.scale}scale.png')